# Bewise.ai

# Тестовое задание на позицию "Junior Data Scientist (NLP)" 

Необходимо написать скрипт для парсинга диалогов из файла test_data.csv.

Главные задачи, которые должен выполнять скрипт:

Извлекать реплики с приветствием – где менеджер поздоровался. 

Извлекать реплики, где менеджер представил себя. 

Извлекать имя менеджера. 

Извлекать название компании. 

Извлекать реплики, где менеджер попрощался.

Проверять требование к менеджеру: «В каждом диалоге обязательно необходимо поздороваться и попрощаться с клиентом»

Рекомендации:

Можно создать дополнительное поле в таблице test_data.csv, куда будет сохраняться результат парсинга – например, напротив реплики в столбце “insight” можно ставить флаг того, что эта реплика с приветствием greeting=True.
Для выполнения задачи можно использовать любые библиотеки и NLP модели. 
Попробуйте учесть возможные синонимичные выражения, которые могут помочь с извлечением данных сущностей. 

Добрый день! Решение тестового задания приведено ниже:

In [1]:
# импорт библиотек
import pandas as pd
import re

pd.set_option('display.max_colwidth', None)  #текст в ячейке отражается полностью

In [2]:
# считаем в переменную файл
test_data = pd.read_csv('test_data.csv')

In [3]:
test_data['dlg_id'].unique()

array([0, 1, 2, 3, 4, 5], dtype=int64)

dlg_id  (dialog_id)- идентификатор диалога менеджера и клиента

line_n - номер реплики в диалоге

role -  роль собеседника, менеджер или клиент

In [4]:
test_data.isna().sum()

dlg_id    0
line_n    0
role      0
text      0
dtype: int64

пропусков нет

In [5]:
test_data.duplicated().sum()

0

явных дубликатов нет

Необходимо учесть, что в компаниях обычно разрабатываются скрипты диалогов менеджеров с клиентами, допустим несколько моментов:

1) Логично сначала поприветствовать клиента (номер реплики приветствия в рамках текущего диалога не должен превышать 3, в середине разговора уже не здороваются)

2) Номер реплики прощания не может быть дальше 3 реплики с конца (условно, а лучше даже 1 реплика с конца у менеджера)

3) Если менеджер представляется, то как правило звучит фраза "меня зовут... ", "это ...", "... зовут",  "я ..." (последний шаблон можно отнести к деловому общению уже формально, в коде я его не буду использовать).  В дальнейшем в работе можно проконсультироваться с авторами скриптов для разговоров с клиентами, а также проанализировать частоты слов, встречающихся в первых репликах диалогов, и "отловить" другие шаблоны знакомства менеджеров, но на данном этапе будем использовать только эти

4) название компании, как правило звучит в диалоге после слова "компания ...", но в данном случае, сложность представляет то, что мы не знаем, из скольки слов состоит название компании. В этом месте предполагаю составить заранее словарь названий компаний, и используя 1 слово после "компания" сопоставлять. Это может не сработать, если у менеджера в речи не прозвучало "компания ...", и если компании начинаются с одинакового слова. В данном тестовом примере данных таких ситуаций нет.

5) для извлечения сущностей приветствия и прощания используем шаблоны реплик, с учетом того, что реплика может начинаться с заглавной буквы

In [6]:
def define_manager_name(data, k):
    """
    Функция определяет имя менеджера в текущем диалоге и индекс реплики, в которой прозвучало имя менеджера
    :param data:        (DataFrame)
                        датафрейм с исходными репликами  
    :param k:           (int)
                        id диалога
    :return number_name, manager_name: номер реплики, имя менеджера
    """
    try:
        # фильтруем первые 4 строки, дальше искать имя менеджера бессмысленно (можно и меньше оставлять строк)
        current = data[data['line_n']<=3]
        current = current[current['role']=='manager']
        #выбираем номер диалога и добавляем шаблоны для поиска имени, учитываем, что мы ищем имя менеджера, а не клиента
        current = current[current['dlg_id']==k]['text'].str.extract(r'зовут (\b\w*\b)|это (\b\w*\b)| (\b\w*\b) зовут') 
        # посмотрим, тк сформировался датафрейм, куда именно попали имена 
        # (тк в зависимости от порядка слова в строке, имя могло попасть в любой столбец), 
        # оставим только ту строчку, где есть имя
        is_null = current.isnull()
        current = current[~is_null.all(axis=1)]
        is_na = current.isna()
        #посмотрим теперь, на каком порядковом месте (определим столбец) оказалось имя (тк остальные значения будут пропущены)
        my_filter = (is_na == False).any()
        #индекс строки, в которой менеджер представился
        id_manager_name = current.index[0] 
        #имя менеджера
        manager_name = current.loc[id_manager_name][ my_filter].values[0]
    except:
        # может возникнуть исключение (если менеджер не представился)
        id_manager_name = 'n \ a' 
        manager_name = 'n \ a'
    return id_manager_name, manager_name

In [7]:
def define_company_name(data, k):
    """
    Функция определяет название компании в текущем диалоге и индекс реплики, в которой отразилось название компании
    :param data:        (DataFrame)
                        датафрейм с исходными репликами  
    :param k:           (int)
                        id диалога
    :return id_number_company, company_name: номер реплики, название компании
    """
    try:
        # заранее опишем возможные синонимы для названий компаний
        company_names = {'диджитал бизнес':['диджитал','бизнес','диджитал бизнес'],'китобизнес':['китобизнес']}
        
        current = data[data['line_n']<=3]
        current = current[current['role']=='manager']
        current = current[current['dlg_id']== k]['text'].str.extract(r'компания (\b\w*\b)') 
        is_null = current.isnull()
        current = current[~is_null.all(axis=1)]
        is_na = current.isna()
        #посмотрим на каком порядковом месте оказалось название (тк остальные значения будут пропущены)
        my_filter = (is_na == False).any()
        #индекс строки, в которой прозвучало название компании
        id_number_company = current.index[0] 
        #название компании
        company_name = current.loc[id_number_company][ my_filter].values[0]
        for name, syn in company_names.items():
                if company_name in syn: company_name = name
    except:
        # может возникнуть исключение (если название компании не прозвучало)
        id_number_company = 'n \ a' 
        company_name = 'n \ a'
    return id_number_company, company_name

In [8]:
def define_greetings(data, k):
    """Функция определяет, поздоровался ли менеджер в текущем диалоге и индекс реплики приветствия
    :param data:        (DataFrame)
                        датафрейм с исходными репликами  
    :param k:           (int)
                        id диалога
    :return greeting, id_greeting: флаг приветствия, id реплики
    """
    try:
        current = data[data['line_n']<=3]
        current = current[current['role']=='manager']
        current = current[current['dlg_id']== k]['text'].str.extract(r'(оброе утро|дравствуйте|обрый день|обрый вечер)') 
        is_null = current.isnull()
        current = current[~is_null.all(axis=1)]
        is_na = current.isna()
        #посмотрим на каком порядковом месте оказалось приветствие
        my_filter = (is_na == False).any()
        if all(my_filter):
            #индекс строки, в которой прозвучало приветствие
            id_greeting = current.index[0] 
            # флаг приветствия
            greeting = True
        else: 
            id_greeting = 'n  \ a'
            greeting = False
    except:
        # если возникнет исключение в случае отсутствия приветствия
        id_greeting = 'n  \ a'
        greeting = 'n  \ a'
    return id_greeting, greeting 

In [9]:
def define_goodbyes(data, k):
    """Функция определяет, попрощался ли менеджер в текущем диалоге и индекс реплики прощания
    :param data:        (DataFrame)
                        датафрейм с исходными репликами  
    :param k:           (int)
                        id диалога
    :return goodbye, id_goodbye: флаг прощания, id реплики
    """
    try:
        current = data[data['dlg_id']==k]
        dlg_len = max(current['line_n'])
        current = current[current['line_n'] >= dlg_len - 3] #берем только последние строчки диалога
        current = current[current['role']=='manager']
        current = current['text'].str.extract(r'(о свидания|сего доброго|орошего вечера|орошего дня|орошего утра)') 
        
        is_null = current.isnull()
        current = current[~is_null.all(axis=1)]
        is_na = current.isna()
        #посмотрим на каком порядковом месте оказалось прощание
        my_filter = (is_na == False).any()
        if all(my_filter):
            #индекс строки, в которой прозвучало прощание
            id_goodbye = current.index[0] 
            # флаг прощания
            goodbye = True
        else: 
            id_goodbye = 'n  \ a'
            goodbye = False
    except:
        # на случай возникновения исключения (не попрощался вообще)
        id_goodbye = 'n  \ a'
        goodbye = 'n  \ a'       
    return id_goodbye, goodbye 

Заведем словарь, значениями которого пока будут пустые списки

In [10]:
politely = {}
politely['dlg_id'] = []
politely['id_manager_name'] = []
politely['manager_name'] = []
politely['greeting'] = []
politely['id_greeting'] = []
politely['goodbye'] = []
politely['id_goodbye'] = []
politely['company_name'] = []
politely['id_company_name']=[]

Заполним словарь:

In [11]:
for i in test_data['dlg_id'].unique():
    number_name, manager_name = define_manager_name(test_data, i)
    id_number_company, company_name = define_company_name(test_data, i)
    id_greeting, greeting = define_greetings(test_data, i)
    id_goodbye, goodbye = define_goodbyes(test_data, i)
    politely['dlg_id'].append(i) 
    politely['id_manager_name'].append(number_name)
    politely['manager_name'].append(manager_name)
    politely['company_name'].append(company_name)
    politely['id_company_name'].append(id_number_company)
    politely['greeting'].append(greeting)
    politely['id_greeting'].append(id_greeting)
    politely['goodbye'].append(goodbye)
    politely['id_goodbye'].append(id_goodbye)

In [12]:
def polite_manager(row):
    """Функция определяет, выполняется ли требование и приветствия, и прощания в диалоге
    :param row:        (DataFrame)
                        строка датафрейма, к которой применяется функция
    :return polite: флаг вежливости
    """
    return row['greeting'] and row['goodbye']

Составим из нашего словаря датафрейм для наглядности.

Данный датафрейм и является результатом работы скрипта, приммененному к тестовым данным.

In [13]:
df = pd.DataFrame(politely)
df['polite_manager'] = df.apply(polite_manager, axis = 1) 
df

,dlg_id,id_manager_name,manager_name,greeting,id_greeting,goodbye,id_goodbye,company_name,id_company_name,polite_manager
0,0,3,ангелина,True,1,True,108,диджитал бизнес,3,True
1,1,111,ангелина,True,110,True,163,диджитал бизнес,111,True
2,2,167,ангелина,True,166,False,n \ a,диджитал бизнес,167,False
3,3,251,максим,True,250,True,300,китобизнес,251,True
4,4,n \ a,n \ a,False,n \ a,True,335,n \ a,n \ a,False
5,5,338,анастасия,False,n \ a,True,479,n \ a,n \ a,False


# Выводы:

Результат парсинга указанных диалогов удалось структурировать в отдельный датафрейм. В нем мы видим, что Ангелина - самый вежливый менеджер, во всех трех диалогах она представилась, в 2 и поздоровалась и попрощалась, и везде озвучила название компании.

Самый некорректный менеджер - это менеджер в диалоге с id = 4, он не поздоровался, не попрощался,  и не назвал компанию.

Максим и Анастасия сработали примерно одинаково, допуская некоторые несовпадающие ошибки (есть, куда расти).

При выполнении тестового задания я использовала книгу Б. Форта "Освой самостоятельно регулярные выражения. 10 минут на урок" и Х. Лейн, Х. Хапке, К. Ховард "Обработка естественного языка в действии", статьи в сети на тему делового общения по телефону, а также элементы кода, написанного мною при выполнении проектов в Яндекс Практикуме.

Данное тестовое задание я выполнила примерно за 1,5 рабочих дня (12 часов), в оставшееся время до 4 сентября планирую подумать над другим решением, используя библиотеку SpaCy.
